# Tutorial: Generación de Mapas de Rejas
## Región Metropolitana - Santiago, Chile

Este notebook te guía paso a paso en la generación de mapas interactivos de rejas.

---

### 📋 Requisitos

Antes de empezar, asegúrate de tener instaladas las librerías necesarias:

In [ ]:
# Instalar dependencias (ejecutar solo una vez)
!pip install pandas folium openpyxl

---

## 📊 Paso 1: Explorar los Datos

Primero vamos a cargar y explorar los datos de rejas:

In [1]:
import pandas as pd

# Cargar datos
df = pd.read_excel('01_datos_originales/Rejas.xlsx')

# Mostrar primeras filas
print("Primeras 5 rejas:")
df.head()

Primeras 5 rejas:


,cord,estado,año
0,"-33.478943, -70.657108",1,2025
1,"-33.478980, -70.657970",1,2025
2,"-33.479006, -70.658871",1,2025
3,"-33.478954, -70.659682",1,2025
4,"-33.478975, -70.660479",1,2025


In [ ]:
# Información general del dataset
print(f"Total de rejas: {len(df)}")
print(f"\nColumnas disponibles: {df.columns.tolist()}")
print(f"\nTipos de datos:")
df.dtypes

In [ ]:
# Estadísticas por estado
print("Distribución por estado:")
print(df['estado'].value_counts())

abiertas = len(df[df['estado'] == 1])
cerradas = len(df[df['estado'] == 0])

print(f"\nRejas ABIERTAS (1): {abiertas} ({abiertas/len(df)*100:.1f}%)")
print(f"Rejas CERRADAS (0): {cerradas} ({cerradas/len(df)*100:.1f}%)")

In [ ]:
# Distribución por año
print("Rejas por año:")
df['año'].value_counts().sort_index()

---

## 🗺️ Opción 1: Script Simple (Recomendado)

### Características:
- ✅ Código muy comentado
- ✅ Funciones modulares
- ✅ Fácil de entender
- ✅ Ideal para compartir con colegas

### Ejecutar el script completo:

In [ ]:
# Ejecutar el script simple
%run 02_scripts/mapa_rejas_SIMPLE.py

### Ver el resultado:

El mapa se ha generado en: `04_mapas_html/Mapa_Rejas_RegionMetropolitana.html`

Puedes abrirlo directamente desde el explorador de archivos o ejecutar:

In [ ]:
import webbrowser
import os

# Abrir el mapa en el navegador
ruta_mapa = os.path.abspath('04_mapas_html/Mapa_Rejas_RegionMetropolitana.html')
webbrowser.open('file://' + ruta_mapa)

print(f"Mapa abierto: {ruta_mapa}")

---

## 🔧 Opción 2: Script Original

### Características:
- Versión más compacta
- Mismo resultado visual
- Menos comentarios

### Ejecutar:

In [ ]:
# Ejecutar el script original
%run 02_scripts/mapa_rejas.py

---

## 📝 Paso a Paso: Entender el Proceso

Si quieres entender cómo funciona el proceso paso a paso, aquí está desglosado:

### Paso 2.1: Procesar Coordenadas

In [ ]:
# Separar coordenadas del formato "lat, lon"
df[['Latitud', 'Longitud']] = df['cord'].str.split(',', expand=True)
df['Latitud'] = df['Latitud'].astype(float)
df['Longitud'] = df['Longitud'].astype(float)

print("Coordenadas procesadas:")
df[['cord', 'Latitud', 'Longitud']].head()

### Paso 2.2: Calcular Centro del Mapa

In [ ]:
# Calcular centro (promedio de coordenadas)
centro_lat = df['Latitud'].mean()
centro_lon = df['Longitud'].mean()

print(f"Centro del mapa:")
print(f"  Latitud: {centro_lat:.6f}")
print(f"  Longitud: {centro_lon:.6f}")

### Paso 2.3: Función de Gradiente de Colores

In [ ]:
def calcular_color_gradiente(año, año_min, año_max):
    """
    Calcula color para rejas cerradas basado en año.
    Oscuro (2012) -> Claro (2025)
    """
    if año_max == año_min:
        gradiente = 0.5
    else:
        gradiente = (año - año_min) / (año_max - año_min)
        gradiente = max(0, min(1, gradiente))
    
    # Interpolación RGB
    r = int(20 + (255 - 20) * gradiente)
    g = int(50 + (255 - 50) * gradiente)
    b = int(100 + (150 - 100) * gradiente * 0.5)
    
    return f'#{r:02x}{g:02x}{b:02x}'

# Probar con algunos años
año_min = df['año'].min()
año_max = df['año'].max()

print("Ejemplos de colores por año:")
for año in [2012, 2015, 2018, 2021, 2025]:
    color = calcular_color_gradiente(año, año_min, año_max)
    print(f"  {año}: {color}")

### Paso 2.4: Crear Mapa Básico

In [ ]:
import folium

# Crear mapa básico
mapa_prueba = folium.Map(
    location=[centro_lat, centro_lon],
    zoom_start=12,
    tiles='OpenStreetMap'
)

# Agregar un marcador de prueba
folium.Marker(
    location=[centro_lat, centro_lon],
    popup="Centro del mapa",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(mapa_prueba)

# Mostrar mapa
mapa_prueba

### Paso 2.5: Agregar Marcadores de Rejas (Muestra)

In [ ]:
# Crear mapa con primeras 20 rejas como ejemplo
mapa_muestra = folium.Map(
    location=[centro_lat, centro_lon],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Agregar solo las primeras 20 rejas
for idx, row in df.head(20).iterrows():
    lat = row['Latitud']
    lon = row['Longitud']
    estado = row['estado']
    año = row['año']
    
    # Color según estado
    if estado == 0:
        color = calcular_color_gradiente(año, año_min, año_max)
    else:
        color = '#e74c3c'  # Rojo para abiertas
    
    # Crear marcador
    folium.CircleMarker(
        location=[lat, lon],
        radius=7,
        popup=f"Reja #{idx+1} - {'ABIERTA' if estado == 1 else 'CERRADA'} ({año})",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.8
    ).add_to(mapa_muestra)

print("Mapa de muestra con 20 rejas:")
mapa_muestra

---

## 📊 Análisis Adicional

### Visualizar distribución por año

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de barras por año
df_por_año = df['año'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
df_por_año.plot(kind='bar', color='steelblue')
plt.title('Distribución de Rejas por Año', fontsize=16, fontweight='bold')
plt.xlabel('Año', fontsize=12)
plt.ylabel('Cantidad de Rejas', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAño con más rejas: {df_por_año.idxmax()} ({df_por_año.max()} rejas)")

### Comparar Abiertas vs Cerradas

In [ ]:
# Gráfico de pastel
estados = df['estado'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(
    estados.values,
    labels=['Abiertas (1)', 'Cerradas (0)'],
    autopct='%1.1f%%',
    colors=['#e74c3c', '#3498db'],
    startangle=90,
    textprops={'fontsize': 12}
)
plt.title('Estado de las Rejas', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

---

## 💾 Exportar Datos Procesados

In [ ]:
# Guardar datos procesados con coordenadas separadas
df.to_csv('01_datos_originales/Rejas_procesado.csv', index=False, encoding='utf-8-sig')
print("Datos procesados guardados en: 01_datos_originales/Rejas_procesado.csv")

---

## 📋 Resumen

### Archivos generados:

1. **04_mapas_html/Mapa_Rejas_RegionMetropolitana.html**
   - Mapa interactivo completo
   - 176 rejas con gradiente de colores
   - Filtros por estado

2. **01_datos_originales/Rejas_procesado.csv** (opcional)
   - Datos con coordenadas procesadas

### Próximos pasos:

- ✅ Abre el mapa HTML en tu navegador
- ✅ Explora las rejas usando los filtros
- ✅ Haz click en los marcadores para ver detalles
- ✅ Comparte el HTML con tu equipo

---

**¿Preguntas?** Revisa el README.md en la raíz del proyecto.